In [ ]:
!pip install -qU datasets
!pip install -qU transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 116.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("matejklemen/clc_fce")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/540 [00:00<?, ?B/s]

clc_fce.py:   0%|          | 0.00/5.20k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.57M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/211k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/244k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2191 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2695 [00:00<?, ? examples/s]

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']
val_dataset = dataset['validation']

In [ ]:
def tokens_to_text(tokens):
    return " ".join(tokens)

def preprocess_data(examples):
    inputs = [tokens_to_text(tokens) for tokens in examples["src_tokens"]]
    targets = [tokens_to_text(tokens) for tokens in examples["tgt_tokens"]]
    return {"input": inputs, "target": targets}

processed_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/28350 [00:00<?, ? examples/s]

Map:   0%|          | 0/2191 [00:00<?, ? examples/s]

Map:   0%|          | 0/2695 [00:00<?, ? examples/s]

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")

def tokenize_function(examples):
    inputs = [f"correct: {text}" for text in examples["input"]]
    targets = examples["target"]
    model_inputs = tokenizer(inputs, text_target=targets, truncation=True, padding=True)
    return model_inputs

tokenized_dataset = processed_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/28350 [00:00<?, ? examples/s]

Map:   0%|          | 0/2191 [00:00<?, ? examples/s]

Map:   0%|          | 0/2695 [00:00<?, ? examples/s]

In [ ]:
from transformers import T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small").to(device)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="epoch",
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],

)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-d78e7252f2ce>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.058600,0.050539
2,0.056000,0.049242
3,0.053900,0.049410
4,0.052200,0.048988
5,0.050700,0.049118
6,0.049300,0.049433
7,0.048300,0.049769


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=24808, training_loss=0.05272183607409593, metrics={'train_runtime': 5541.8479, 'train_samples_per_second': 51.156, 'train_steps_per_second': 6.395, 'total_flos': 1.097526222096384e+16, 'train_loss': 0.05272183607409593, 'epoch': 7.0})

In [ ]:
from google.colab import files
trainer.save_model("my_trained_model")

!zip -r my_trained_model.zip my_trained_model

files.download('my_trained_model.zip')

updating: my_trained_model/ (stored 0%)
updating: my_trained_model/spiece.model (deflated 48%)
updating: my_trained_model/training_args.bin (deflated 52%)
updating: my_trained_model/model.safetensors (deflated 7%)
updating: my_trained_model/tokenizer_config.json (deflated 94%)
updating: my_trained_model/config.json (deflated 62%)
updating: my_trained_model/generation_config.json (deflated 29%)
updating: my_trained_model/special_tokens_map.json (deflated 85%)
updating: my_trained_model/added_tokens.json (deflated 83%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
input_text = "I am writing in order to express my disappointment about your musical show 'Over the Rainbow'."
inputs = tokenizer(f"correct: {input_text}", return_tensors="pt").to(device)
outputs = model.generate(**inputs)
corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(corrected_text)  # Çıktı: "I am writing in order to express my disappointment with your musical show 'Over the Rainbow'."

I am writing in order to express my disappointment with your musical show 'Over the Rainbow'
